# **WIDS Datathon : Raphael Yao**

### **Loading Files, Packages, Scripts:**

In [ ]:
!pip install scikit-learn
!pip install --quiet optuna
!pip install category_encoders
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, r2_score, mean_squared_error

from sklearn.linear_model import LogisticRegression, LinearRegression

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.cluster import KMeans

import optuna
import warnings
import scipy
import category_encoders
from category_encoders import OneHotEncoder, MEstimateEncoder, CatBoostEncoder, OrdinalEncoder

from sklearn import set_config
from sklearn import linear_model
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_val_score, KFold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, f1_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import FunctionTransformer,StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Ridge
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, ExtraTreesRegressor
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier, VotingRegressor
from sklearn.svm import SVC
from sklearn.metrics import auc, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, Pool, CatBoostRegressor

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.cluster import KMeans


pd.set_option('display.max_rows', 150)

In [ ]:
train = pd.read_csv(r'/content/training.csv', index_col='patient_id')
test = pd.read_csv(r'/content/test.csv', index_col='patient_id')
sub  = pd.read_csv(r'/content/sample_submission.csv')

### **Initial Data : Scraping and Observations:**

In [ ]:
desc = pd.DataFrame(index = list(train))
desc['type'] = train.dtypes
desc['count'] = train.count()
desc['nunique'] = train.nunique()
desc['%unique'] = desc['nunique'] /len(train) * 100
desc['null'] = train.isnull().sum()
desc['%null'] = desc['null'] / len(train) * 100
desc = pd.concat([desc,train.describe().T.drop('count',axis=1)],axis=1)
desc.sort_values(by=['type','null']).style.background_gradient(axis=0)

,type,count,nunique,%unique,null,%null,mean,std,min,25%,50%,75%,max
patient_zip3,int64,12906,739,5.726019,0,0.000000,573.754300,275.447534,101.000000,331.000000,554.000000,846.000000,999.000000
patient_age,int64,12906,67,0.519138,0,0.000000,59.183326,13.335216,18.000000,50.000000,59.000000,67.000000,91.000000
DiagPeriodL90D,int64,12906,2,0.015497,0,0.000000,0.624516,0.484266,0.000000,0.000000,1.000000,1.000000,1.000000
population,float64,12905,738,5.718271,1,0.007748,20744.441237,13886.903756,635.545455,9463.896552,19154.190480,30021.278690,71374.131580
density,float64,12905,738,5.718271,1,0.007748,1581.950419,2966.305306,0.916667,171.857143,700.337500,1666.515385,21172.000000
age_median,float64,12905,728,5.640787,1,0.007748,40.502259,4.036963,20.600000,37.129825,40.639344,42.934783,54.570000
age_under_10,float64,12905,718,5.563304,1,0.007748,11.122784,1.512376,0.000000,10.160000,11.039216,12.190000,17.675000
age_10_to_19,float64,12905,725,5.617542,1,0.007748,12.945265,1.923974,6.314286,11.741176,12.923944,14.019767,35.300000
age_20s,float64,12905,724,5.609794,1,0.007748,13.290376,3.354103,5.925000,11.013415,12.538095,14.971053,62.100000
age_30s,float64,12905,725,5.617542,1,0.007748,12.862036,2.324922,1.500000,11.292105,12.442647,14.026667,25.471429


We can make a few observations:
*   Most demographic variables are related to patient_zip3 and population ; that has 739 unique Zip codes
*   Certain variables have insufficient data to be imputed and must be deleted:
  * `metastatic_first_novel_treatment, metastatic_first_novel_treatment_type, patient_type, patient_race, and bmi`
*   I would like to also remove any patients with insufficient data because if they have a lot of missing column inputs then their data is unhealthy
*   For data cleaning, we are going to `remove patient_state, Region, Division`
  * I feel that the Zip3 (first 3 digits) provides sufficient evidence; it may overfit
*   I am then going to put the variables through a trasnformer pipeline to transform them.




## **Cleaning and Preprocessing Data:**

### Finding Null Rows

In [ ]:
#Finding null rows

nulls_row = train.isnull().sum(axis=1)

nulls_freq = nulls_row.value_counts().sort_index()

nulls_table = pd.DataFrame({'Null # Frequency': nulls_freq.index,
                                               'Frequency': nulls_freq.values})
nulls_table

,Null # Frequency,Frequency
0,0,3
1,1,9
2,2,1661
3,3,5771
4,4,4907
5,5,477
6,6,40
7,7,26
8,8,7
9,10,1


In [ ]:
#Let's delete rows with Nulls >= 5 (Null > 4). It will clean up the noise in the data

null_thresh = 4
null_row_4 = train[train.isnull().sum(axis=1) > null_thresh]
train_clean = train.drop(null_row_4.index)
train_clean

,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
patient_id,,,,,,,,,,,,,,,,,,,,,
475714,NaN,MEDICAID,CA,924,84,F,NaN,C50919,Malignant neoplasm of unsp site of unspecified...,C7989,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
349367,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
138632,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,C773,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
617843,White,COMMERCIAL,CA,926,45,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,C773,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
817482,NaN,COMMERCIAL,ID,836,55,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",C773,...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674178,White,NaN,OH,436,50,F,32.11,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,...,17.400000,23.600000,0.864706,19.841176,6.300000,6.247059,38.753055,8.068682,21.140731,1
452909,NaN,COMMERCIAL,CA,945,50,F,NaN,C50912,Malignant neoplasm of unspecified site of left...,C773,...,11.243210,7.837037,5.411250,34.700000,3.845679,5.671605,36.469947,6.265266,10.728732,1
357486,NaN,COMMERCIAL,CA,926,61,F,29.24,C50912,Malignant neoplasm of unspecified site of left...,C7931,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,1


In [ ]:
nulls_row1 = train_clean.isnull().sum(axis=1)

nulls_freq1 = nulls_row1.value_counts().sort_index()

nulls_table1 = pd.DataFrame({'Null # Frequency': nulls_freq1.index,
                                               'Frequency': nulls_freq1.values})
nulls_table1

,Null # Frequency,Frequency
0,0,3
1,1,9
2,2,1661
3,3,5771
4,4,4907


### Null Columns and Removing Other Variables

In [ ]:
# Calculate percentage of null values in each column
null_column = (train_clean.isnull().sum() / len(train)) * 100

# Create DataFrame to store column-wise null percentages
null_percentage_df = pd.DataFrame({'Column': null_column.index,
                                   'Null Percentage': null_column.values}).sort_values(by='Null Percentage', ascending=False)

# Print DataFrame
null_percentage_df.head(10)

,Column,Null Percentage
11,metastatic_first_novel_treatment_type,95.513715
10,metastatic_first_novel_treatment,95.513715
6,bmi,65.326205
0,patient_race,45.552456
1,payer_type,10.134821
46,income_household_six_figure,0.000000
53,education_less_highschool,0.000000
60,labor_force_participation,0.000000
59,education_stem_degree,0.000000
58,education_college_or_above,0.000000


In [ ]:
#removing anything that is over 10% null
null_thresh = 10
null_columns = train_clean.columns[train_clean.isnull().mean() * 100 > null_thresh]
train_clean = train_clean.drop(null_columns, axis = 1)

#removing state, region, division
train_clean = train_clean.drop(['patient_state', 'Region', 'Division'], axis=1)
train_clean

,patient_zip3,patient_age,patient_gender,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,population,density,age_median,age_under_10,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
patient_id,,,,,,,,,,,,,,,,,,,,,
475714,924,84,F,C50919,Malignant neoplasm of unsp site of unspecified...,C7989,31437.75000,1189.562500,30.642857,16.014286,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
349367,928,62,F,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,39121.87879,2295.939394,38.200000,11.878788,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
138632,760,43,F,C50112,Malignant neoplasm of central portion of left ...,C773,21996.68333,626.236667,37.906667,13.028333,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
617843,926,45,F,C50212,Malig neoplasm of upper-inner quadrant of left...,C773,32795.32558,1896.220930,42.871429,10.071429,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
817482,836,55,F,1749,"Malignant neoplasm of breast (female), unspeci...",C773,10886.26000,116.886000,43.473469,10.824000,...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674178,436,50,F,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,19413.05882,1196.805882,36.911765,12.876471,...,17.400000,23.600000,0.864706,19.841176,6.300000,6.247059,38.753055,8.068682,21.140731,1
452909,945,50,F,C50912,Malignant neoplasm of unspecified site of left...,C773,30153.87952,976.289157,42.135802,10.753086,...,11.243210,7.837037,5.411250,34.700000,3.845679,5.671605,36.469947,6.265266,10.728732,1
357486,926,61,F,C50912,Malignant neoplasm of unspecified site of left...,C7931,32795.32558,1896.220930,42.871429,10.071429,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,1


In [ ]:
train_clean['patient_zip3'] = train_clean['patient_zip3'].astype('object')

In [ ]:
train_clean_columns = train_clean.drop('DiagPeriodL90D', axis=1).columns

# Align test to match the columns of train_clean
test_clean = test[train_clean_columns]

test_clean['patient_zip3'] = test_clean['patient_zip3'].astype('object')

# Now test_clean has the same columns as train_clean
test_clean

<ipython-input-13-c320c0226f9e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_clean['patient_zip3'] = test_clean['patient_zip3'].astype('object')


,patient_zip3,patient_age,patient_gender,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,population,density,age_median,age_under_10,...,hispanic,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02
patient_id,,,,,,,,,,,,,,,,,,,,,
573710,467,54,F,C50412,Malig neoplasm of upper-outer quadrant of left...,C773,5441.435484,85.620968,40.880328,12.732258,...,3.564516,13.996774,7.985484,0.969355,24.955357,10.838710,8.080645,38.724876,7.947165,11.157161
593679,337,52,F,C50912,Malignant neoplasm of unspecified site of left...,C787,19613.820510,1555.107692,49.107692,8.069231,...,10.261538,16.020513,13.602564,2.836842,23.952632,10.579487,9.302564,36.918257,7.838973,13.599985
184532,917,61,F,C50911,Malignant neoplasm of unsp site of right femal...,C773,43030.500000,2048.578261,38.852174,11.306522,...,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
447383,917,64,F,C50912,Malignant neoplasm of unspecified site of left...,C779,43030.500000,2048.578261,38.852174,11.306522,...,47.726087,9.895652,10.515217,12.745652,32.530435,7.263043,3.810870,47.310325,9.595719,20.084231
687972,900,40,F,C50412,Malig neoplasm of upper-outer quadrant of left...,C779,36054.117650,5294.330882,36.653846,9.761538,...,45.526154,11.901538,20.760000,14.737500,30.709375,10.341538,3.030769,41.186992,11.166898,21.644261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977076,404,63,F,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,7765.000000,131.040000,40.434783,11.556522,...,1.243478,20.404348,20.813636,0.350000,30.152174,6.473913,5.908696,39.947326,7.622672,9.154618
922960,507,69,F,C50912,Malignant neoplasm of unspecified site of left...,C773,19332.750000,346.250000,38.525000,12.200000,...,6.175000,16.675000,15.900000,2.800000,16.800000,5.475000,6.875000,35.825340,7.610534,9.712786
759690,980,84,F,C50411,Malig neoplm of upper-outer quadrant of right ...,C773,28628.290910,1091.825455,39.679245,12.143396,...,9.511321,9.752830,6.432075,5.094340,31.275472,5.309434,5.807547,36.618644,4.939852,23.393650


## **Pipeline and Modeling Data**

In [ ]:
numeric_features = train_clean.select_dtypes('number').drop(columns = ["DiagPeriodL90D"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_features = [c for c in train_clean.columns if c not in numeric_features and c != 'DiagPeriodL90D']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
y = train_clean.DiagPeriodL90D
X = train_clean.drop('DiagPeriodL90D', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [ ]:
model.fit(X_train, y_train,)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Training Accuracy: 0.832991471445536
Testing Accuracy: 0.8144430051813472


In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 669,  495],
       [  78, 1846]])

In [ ]:
# prompt: find and save patient_id from test_clean

test_clean_patient_id = test_clean.index.to_list()

In [ ]:
predictions = model.predict(test_clean)
submission_dataframe = pd.DataFrame({'patient_id':test_clean_patient_id,'Predictions':predictions})
submission_dataframe.to_csv('firstsubmission.csv', index=False)

In [ ]:
knn_r_acc = []  # Initialize list to store results

# Loop through different values of n_neighbors
for i in range(1, 17):
    # Create KNeighborsRegressor with current value of n_neighbors
    knn = KNeighborsRegressor(n_neighbors=i)
    model = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())])
    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Calculate test and train scores
    test_score = model.score(X_test, y_test)
    train_score = model.score(X_train, y_train)

    # Append results to the list
    knn_r_acc.append((i, test_score, train_score))

# Create DataFrame from the results
df = pd.DataFrame(knn_r_acc, columns=['K', 'Test Score', 'Train Score'])
print(df)

## CatBoost:

### CatboostClassifier

In [ ]:
import catboost as ctb

In [ ]:
numeric_features = train_clean.select_dtypes('number').drop(columns = ["DiagPeriodL90D"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_features = [c for c in train_clean.columns if c not in numeric_features and c != 'DiagPeriodL90D']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
y = train_clean.DiagPeriodL90D
X = train_clean.drop('DiagPeriodL90D', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('classifier', CatBoostClassifier())
])
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

Training Accuracy: 0.7187736154593545
Testing Accuracy: 0.6295336787564767


In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 247,  917],
       [ 227, 1697]])

### CatBoost Regressor

In [ ]:
numeric_features = train_clean.select_dtypes('number').drop(columns = ["DiagPeriodL90D"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_features = [c for c in train_clean.columns if c not in numeric_features and c != 'DiagPeriodL90D']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
y = train_clean.DiagPeriodL90D
X = train_clean.drop('DiagPeriodL90D', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
model = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor())
])
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
kf = KFold(n_splits=4, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
cv_scores = -cv_scores

print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", np.mean(cv_scores))
print("Std Dev CV Score:", np.std(cv_scores))

In [ ]:
predictions = model.predict(test_clean)
submission2_dataframe = pd.DataFrame({'patient_id':test_clean_patient_id,'Predictions':predictions})
submission2_dataframe.to_csv('submission4.csv', index=False)
submission2_dataframe

,patient_id,Predictions
0,573710,0.952676
1,593679,0.771901
2,184532,0.744157
3,447383,0.754592
4,687972,0.821515
...,...,...
5787,977076,0.807771
5788,922960,0.714920
5789,759690,0.884620
5790,911717,0.081286


## Big Boi Stuff

In [ ]:
numeric_features = train_clean('number').drop(columns = ["DiagPeriodL90D"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_features = [c for c in train_clean.columns if c not in numeric_features and c != 'DiagPeriodL90D']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:
SEED = 42

models = [
           ('lgbm',make_pipeline(preprocessor,
                                 LGBMRegressor(random_state=SEED,
                                               force_col_wise = True))),

           ('RF',make_pipeline(preprocessor,
                               RandomForestRegressor(n_estimators=200,
                                                      random_state=SEED,
                                                      min_samples_leaf=92,
                                                      max_features=1.0))),
           ('Extratrees',make_pipeline(preprocessor,
                                       ExtraTreesRegressor(n_estimators=300,
                                                           random_state=SEED,
                                                           min_samples_leaf=46,
                                                           max_features=1.0))),
           ('XGB',make_pipeline(preprocessor,
                                 XGBRegressor(random_state=SEED))),

           ('catBoost',make_pipeline(preprocessor,
                                    CatBoostRegressor(random_state=SEED,
                                                      logging_level = 'Silent'))),


        ]

In [ ]:
def score_model_regressor(model, label, train_clean, SEED=42):

    print('model: {}'.format(label))
    y = train_clean['DiagPeriodL90D']
    X = train_clean.drop('DiagPeriodL90D', axis=1)

    kf = KFold(n_splits=5, random_state=SEED, shuffle=True)

    val_predictions = np.zeros(len(train_clean))
    score_list = []
    for fold, (trx_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train = X.iloc[trx_idx]
        y_train = y.iloc[trx_idx]
        X_val = X.iloc[val_idx]
        y_val = y.iloc[val_idx]

        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_val = model.predict(X_val)

        mse_train = mean_squared_error(y_train, y_pred_train)
        mse_val = mean_squared_error(y_val, y_pred_val)

        val_predictions[val_idx] = y_pred_val
        score_list.append(mse_val)
        print(f'fold: {fold} - MSE Train: {mse_train} - MSE Val {mse_val}')

    print(f'MSE MEAN {np.mean(score_list)} - Std: {np.std(score_list)}')

    return score_list, val_predictions

In [ ]:
score_list, oof_list = pd.DataFrame(), pd.DataFrame()

for label, model in models:
    score_list[label], oof_list[label] = score_model_regressor(model, label, train_clean)

model: lgbm
[LightGBM] [Info] Total Bins 17355
[LightGBM] [Info] Number of data points in the train set: 9880, number of used features: 292
[LightGBM] [Info] Start training from score 0.621964
fold: 0 - MSE Train: 0.11037788349946281 - MSE Val 0.15220942281602876
[LightGBM] [Info] Total Bins 17337
[LightGBM] [Info] Number of data points in the train set: 9881, number of used features: 284
[LightGBM] [Info] Start training from score 0.622305
fold: 1 - MSE Train: 0.10938558544610372 - MSE Val 0.15656621157164338
[LightGBM] [Info] Total Bins 17354
[LightGBM] [Info] Number of data points in the train set: 9881, number of used features: 289
[LightGBM] [Info] Start training from score 0.622811
fold: 2 - MSE Train: 0.1116696511643045 - MSE Val 0.14874497299724998
[LightGBM] [Info] Total Bins 17338
[LightGBM] [Info] Number of data points in the train set: 9881, number of used features: 279
[LightGBM] [Info] Start training from score 0.623216
fold: 3 - MSE Train: 0.10870374869043214 - MSE Val 0

In [ ]:
ridge_model = Ridge()
ridge_model.fit(oof_list, train_clean['DiagPeriodL90D'])
w = ridge_model.coef_
w

array([-0.00914904,  0.00756961,  0.66561555, -0.0023259 ,  0.3276396 ])

In [ ]:
from sklearn.ensemble import VotingRegressor

In [ ]:
voter = VotingRegressor(models, weights=w)

voter.fit(X, y)

In [ ]:
sub["DiagPeriodL90D"] = voter.predict(test_clean)
sub.to_csv('submission11.csv',index=False)